In [3]:
import sys,os
__script_path=os.path.abspath(globals().get('__file__','.'))
__script_dir = os.path.dirname(__script_path)
root_dir = os.path.abspath(f'{__script_dir}/..')
print(root_dir)
for lib in [root_dir][::-1]:
    if lib in sys.path:
        sys.path.remove(lib)
    sys.path.insert(0,lib)


c:\Users\PC\Data\multimodal-rag-baseline


In [4]:
from configs.config import *
from libs.common import *
from utils.mongodb_connection import *
from utils.rag_utils import *
from utils.format_utils import *
load_dotenv(find_dotenv())

c:/Users/PC/Data/multimodal-rag-baseline


c:\ProgramData\miniconda3\envs\kyanon_digital\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

In [5]:
mongo_uri = os.getenv('MONGO_URI')
GEMINI_API_KEY = os.getenv('GEMINI_API_KEY')
collection = "kyanon_digital"

mongo_client = get_mongo_client(mongo_uri)
db = mongo_client['Kyanon']
collection = db['RAG']

Connection to MongoDB successful


In [6]:
# https://huggingface.co/thenlper/gte-large
embedding_model = SentenceTransformer("thenlper/gte-large")
embedding_model = embedding_model.to('cuda')

print("Model is using GPU:", next(embedding_model.parameters()).is_cuda)

Model is using GPU: True


In [7]:
import camelot

pdf_path = "content.pdf"

tables = camelot.read_pdf(pdf_path, pages="all", flavor='lattice')
print(f"Tổng cộng: {tables.n} bảng được tìm thấy.")

table_strings = []

for i, table in enumerate(tables):
    table_str = table.df.to_markdown(index=False)  # Dễ đọc hơn với index=False
    table_strings.append(table_str)
    print(f"\n--- Bảng {i + 1} ---")
    print(table_str)



c:\ProgramData\miniconda3\envs\kyanon_digital\lib\site-packages\pypdf\_crypt_providers\_cryptography.py:32: CryptographyDeprecationWarning: ARC4 has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.ARC4 and will be removed from cryptography.hazmat.primitives.ciphers.algorithms in 48.0.0.
  from cryptography.hazmat.primitives.ciphers.algorithms import AES, ARC4


Tổng cộng: 10 bảng được tìm thấy.

--- Bảng 1 ---
| 0   | 1                                                 |
|:----|:--------------------------------------------------|
|     | Elected to the Hall of Fame on this ballot (named |
|     | in bold italics).                                 |
|     | Elected subsequently, as of 2025 (named in plain  |
|     | italics).                                         |
|     | Renominated for the 2019 BBWAA election by        |
|     | adequate performance on this ballot and has not   |
|     | subsequently been eliminated.                     |
|     | Eliminated from annual BBWAA consideration by     |
|     | poor performance or expiration on subsequent      |

--- Bảng 2 ---
| 0   | 1                                                 |
|:----|:--------------------------------------------------|
|     | ballots.                                          |
|     | Eliminated from annual BBWAA consideration by     |
|     | poor performance or expira

In [14]:
import requests
import json

def generate_response(prompt):
    """
    Gửi prompt đến Ollama và nhận về một đoạn phản hồi hoàn chỉnh.
    
    Tham số:
        prompt (str): Câu hỏi hoặc yêu cầu bạn muốn gửi đến mô hình.

    Trả về:
        str: Phản hồi hoàn chỉnh từ mô hình.
    """
    full_response = ""
    context = f"""
    You are given a single table fragment extracted from a PDF document using OCR. This fragment may be part of a larger table that was split across multiple pages due to formatting or page layout.
    Your task is to generate a detailed and comprehensive summary of the content in this table fragment.
    The summary should clearly describe:
        The main subject or topic of the table
        Key columns and their meanings
        Important patterns, trends, or observations in the data
        Any notable values or anomalies
        Contextual information needed to understand the data
    Write the summary in 1–3 full sentences , using clear and precise language.
    If applicable, mention that this fragment appears to be part of a larger table, and include any inferred continuity from the data.
    {prompt}
    """
    url = "http://localhost:11434/api/generate"
    payload = {
        "model": "gemma:latest",
        "prompt": context,
        "stream": True
    }

    with requests.post(url, json=payload, stream=True) as response:
        for line in response.iter_lines():
            if line:
                try:
                    data = json.loads(line)
                    if 'response' in data:
                        chunk = data['response']
                        full_response += chunk
                    if data.get('done', False):
                        break
                except json.JSONDecodeError:
                    continue  

    return full_response.strip()

In [15]:
for i in tables:
    print(i.df)
    table_summary = generate_response(i.df.to_markdown(index=False))
    print(table_summary)
    embedding = get_embedding(embedding_model,table_summary)
    
    chunk_tables = []
    
    document = {
        "embedding": embedding,
        "content": {
            "table": i.df.to_markdown(index=False),
            "summary": table_summary
        }
            
    }
    collection.insert_one(document)

  0                                                  1
0    Elected to the Hall of Fame on this ballot (na...
1    Elected subsequently, as of 2025 (named in pla...
2    Renominated for the 2019 BBWAA election by\nad...
3    Eliminated from annual BBWAA consideration by\...
This table fragment documents the selection process for the Baseball Hall of Fame, highlighting the performance of candidates over multiple years. Key columns include the candidate's name and their status in various elections. Notable observations are the inclusion of bold italics for inaugural inductees and the subsequent elimination of candidates based on poor performance or expiration. This fragment appears to be part of a larger table, suggesting that additional rows exist with further details about the nominated candidates.
   0                                                  1
0                                              ballots.
1     Eliminated from annual BBWAA consideration by\...
2  †                  

In [ ]:
all_tables_combined = ""

for i, table in enumerate(tables):
    table_str = table.df.to_markdown(index=False)  
    all_tables_combined += f"\n\n--- Bảng {i + 1} ---\n"
    all_tables_combined += table_str



print("\n--- Toàn bộ bảng đã gộp ---")
print(all_tables_combined)

In [ ]:
query = "What is The BBWAA election rules?"
source_information = get_search_result(embedding_model, query.lower(), collection)
source_information

In [ ]:
combined_information = f"Become an expert on the concepts on Wikipedia. Customer's question: {query}\nAnswer the question based on the following document information: {source_information}."
print('==== LLMs Inference')
print(combined_information)

In [ ]:
from huggingface_hub import login

api_token = os.getenv('HUGGINGE_FACE_TOKEN') 
login(api_token)



In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("google/gemma-2b-it")
model = AutoModelForCausalLM.from_pretrained("google/gemma-2b-it", device_map="auto")

In [ ]:
input_ids = tokenizer(combined_information, return_tensors="pt").to("cuda")
response = model.generate(**input_ids, max_new_tokens=100)
print(tokenizer.decode(response[0]))